## CONTENT BASED FILTERING RECOMMENDATION SYSTEM 

### EUCLIDEAN DISTANCE GENERATOR 

#### LOAD THE LIBRARIES FOR MODEL

In [1]:
from __future__ import print_function
import pandas as pd
import re
import os
import time
import matplotlib.pyplot as plt
import numpy as np
import collections
from mpl_toolkits.mplot3d import Axes3D
from IPython import display
from matplotlib import pyplot as plt
import sklearn
import sklearn.manifold
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
tf.logging.set_verbosity(tf.logging.ERROR)
pd.options.display.max_columns = 80

2023-06-07 10:20:41.241361: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-07 10:20:41.278099: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-07 10:20:41.463917: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-07 10:20:41.465215: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-07 10:20:42.545295: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

Instructions for updating:
non-resource variables are not supported in the long term


#### LOADING THE DATA AND DATA PRELIMINARY CLEANING 

In [7]:
Moviesdf = pd.read_csv ("imdb_processed.csv")
Moviesdf.head()

,Name,Year,kind,genre,imdb_rating,vote,country,language,cast,director,composer,writer,runtime
0,Dinosaur Planet,2003,tv mini series,"['Documentary', 'Animation', 'Family']",7.7,474.0,['United States'],['English'],"['Christian Slater', 'Scott Sampson']",NaN,NaN,"['Mike Carrol', 'Mike Carroll', 'Georgann Kane']",50.0
1,Get Up and Dance!,1994,video movie,['Family'],8.1,18.0,['United States'],['English'],"['Paula Abdul', 'Aurorah Allain', 'Bill Bohl',...",['Steve Purcell'],NaN,NaN,83.0
2,8 Man,1992,movie,"['Action', 'Sci-Fi']",5.5,93.0,['Japan'],['Japanese'],"['Kai Shishido', 'Etsushi Takahashi', 'Sachiko...",['Yasuhiro Horiuchi'],['Carole King'],"['Kazumasa Hirai', 'Jirô Kuwata', 'Junko Suzuki']",NaN
3,What the #$*! Do We (K)now!?,2004,movie,"['Documentary', 'Comedy', 'Drama', 'Fantasy', ...",5.3,13432.0,['United States'],"['English', 'German', 'Spanish']","['Marlee Matlin', 'Elaine Hendrix', 'John Ross...","['William Arntz', 'Betsy Chasse', 'Mark Vicente']",['Christopher Franke'],"['William Arntz', 'Betsy Chasse', 'Matthew Hof...",60.0
4,Class of Nuke 'Em High Part II: Subhumanoid Me...,1991,movie,"['Comedy', 'Horror', 'Sci-Fi']",4.5,2177.0,['United States'],['English'],"['Brick Bronsky', 'Lisa Gaye', 'Leesa Rowland'...",['Eric Louzil'],['Bob Mithoff'],"['Lloyd Kaufman', 'Carl Morano', 'Matt Unger']",96.0


In [9]:
## Output of the number of rows and columns respectively.
Moviesdf.shape

(8451, 13)

In [10]:
## The information on the dataset
Moviesdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8451 entries, 0 to 8450
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Name         8451 non-null   object 
 1   Year         8451 non-null   int64  
 2   kind         8451 non-null   object 
 3   genre        8384 non-null   object 
 4   imdb_rating  8135 non-null   float64
 5   vote         8135 non-null   float64
 6   country      8152 non-null   object 
 7   language     8052 non-null   object 
 8   cast         8280 non-null   object 
 9   director     7140 non-null   object 
 10  composer     5722 non-null   object 
 11  writer       7204 non-null   object 
 12  runtime      6798 non-null   float64
dtypes: float64(3), int64(1), object(9)
memory usage: 858.4+ KB


In [11]:
## Missing information within the dataframe
Moviesdf.isnull().sum()

Name              0
Year              0
kind              0
genre            67
imdb_rating     316
vote            316
country         299
language        399
cast            171
director       1311
composer       2729
writer         1247
runtime        1653
dtype: int64

In [13]:
# Drop rows with missing values
Moviesdf = Moviesdf.dropna()
Moviesdf.isnull().sum()

Name           0
Year           0
kind           0
genre          0
imdb_rating    0
vote           0
country        0
language       0
cast           0
director       0
composer       0
writer         0
runtime        0
dtype: int64

In [16]:
Moviesdf.shape

(4133, 13)

## CANDIDATE GENERATION 

In [37]:
## Libaries necessary for the function 
from scipy.spatial.distance import euclidean
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler

In [44]:
### Helping functions for model 
## Splits a DataFrame into training and test sets
def split_dataframe(df, holdout_fraction=0.2):
    total_size = len(df)
    test_size = int(total_size * holdout_fraction)
    
    # Generate a random boolean array to select rows for the test set
    test_indices = np.random.choice(total_size, size=test_size, replace=False)
    
    # Split the DataFrame into training and test sets based on the selected indices
    test = df.iloc[test_indices]
    train = df.drop(df.index[test_indices])
    
    return train, test

In [45]:
## the libraries necessary for the euclidean distance calculation 
# Select relevant columns for similarity calculation (e.g., movie features)
selected_columns = ['kind', 'genre', 'country','language']  # Replace with actual feature names
features = Moviesdf[selected_columns]

In [46]:
# Split the data into training and test sets
training_data, test_data = split_dataframe(Moviesdf, holdout_fraction=0.2)

In [47]:
def euclidean_similarity_generator(training_data, features, test_data, k=5):
    # Select features for similarity calculation
    training_features = training_data[features]
    test_features = test_data[features]
    
    # Combine categorical features into a single text column
    training_data['combined_features'] = training_features.apply(lambda x: ' '.join(x), axis=1)
    test_data['combined_features'] = test_features.apply(lambda x: ' '.join(x), axis=1)
    
    # TF-IDF encoding
    vectorizer = TfidfVectorizer()
    training_matrix = vectorizer.fit_transform(training_data['combined_features'])
    test_matrix = vectorizer.transform(test_data['combined_features'])
    
    # Standardize the encoded features
    scaler = StandardScaler(with_mean=False)
    training_features = scaler.fit_transform(training_matrix)
    test_features = scaler.transform(test_matrix)
    
    # Calculate pairwise distances between training and test features
    distances = []
    for i in range(len(test_features)):
        test_movie = test_features[i]
        movie_distances = []
        for j in range(len(training_features)):
            training_movie = training_features[j]
            distance = euclidean(test_movie, training_movie)
            movie_distances.append((distance, j))
        distances.append(movie_distances)
    
    # Sort distances and retrieve indices of most similar movies
    similar_movies_indices = []
    for movie_distances in distances:
        movie_distances.sort()
        indices = [index for _, index in movie_distances[:k]]
        similar_movies_indices.append(indices)
    
    # Get similar movies' information from training data
    similar_movies = [training_data.iloc[indices] for indices in similar_movies_indices]
    
    return similar_movies

In [48]:
# Generate similar movies for test data
# Print the similar movies for each test sample
similar_movies = euclidean_similarity_generator(training_data, features, test_data, k=5)
for i, similar_movie_set in enumerate(similar_movies):
    print(f"Similar movies for test sample {i + 1}:")
    print(similar_movie_set)
    print()

ValueError: Boolean array expected for the condition, not object